In [1]:
# @title Installing Dependencies
!pip install opendatasets rasterio --quiet

In [2]:
# @title Dataset-Loading
import opendatasets as od
od.download("https://www.kaggle.com/datasets/prajwalmohapatra/datasets/data")

Skipping, found downloaded files in "./datasets" (use force=True to force download)


In [3]:
# @title Dependencies
import rasterio
import numpy as np
import os
import glob
import logging

from rasterio.warp import reproject, Resampling

In [4]:
# @title Directory Setup
# Paths
dir = '/home/swayam/projects/forest_fire_spread/datasets'
base_dir = os.path.join(dir, 'dataset_unstacked')
weather_dir = os.path.join(base_dir, 'weather')
fire_mask_dir = os.path.join(base_dir, 'fire_mask')
output_dir = os.path.join(dir, 'dataset_stacked')

print(dir)
print(base_dir)
print(weather_dir)
print(fire_mask_dir)

# Static layers
slope_path = os.path.join(base_dir, 'slope', 'uk_slope_2016.tif')
aspect_path = os.path.join(base_dir, 'aspect', 'uk_aspect_2016.tif')
fuel_path = os.path.join(base_dir, 'fuel_map', 'uk_fuel_map_2020.tif')
ghsl_path = os.path.join(base_dir, 'ghsl', 'ghsl_uk_urbanmask2015.tif')

print()
print(slope_path)
print(aspect_path)
print(fuel_path)
print(ghsl_path)

# Create output directory
os.makedirs(output_dir, exist_ok=True)

/home/swayam/projects/forest_fire_spread/datasets
/home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked
/home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/weather
/home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/fire_mask

/home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/slope/uk_slope_2016.tif
/home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/aspect/uk_aspect_2016.tif
/home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/fuel_map/uk_fuel_map_2020.tif
/home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/ghsl/ghsl_uk_urbanmask2015.tif


In [5]:
# @title Loading Static Layers
# Load static layers once, outside the loop
with rasterio.open(slope_path) as s:
  slope = s.read(1, masked=True)
  ref_meta = s.meta.copy() # Reference metadata
  ref_shape = s.shape # Reference shape (height, width) (9551, 12917)
  ref_transform = s.transform
  ref_crs = s.crs

  print(ref_meta)
  # print(ref_shape)
  # print(ref_transform)
  # print(ref_crs)

with rasterio.open(aspect_path) as a:
  aspect = a.read(1, masked=True)
  # print(aspect)
  print(a.meta.copy())

with rasterio.open(fuel_path) as f:
  fuel = f.read(1, masked=True)
  # print(fuel)
  print(f.meta.copy())

with rasterio.open(ghsl_path) as g:
  ghsl = g.read(1, masked=True)
  # print(ghsl)
  print(g.meta.copy())

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 12917, 'height': 9551, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00026949458523585647, 0.0, 77.56161960922043,
       0.0, -0.00026949458523585647, 31.290207807979588)}
{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 12917, 'height': 9551, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00026949458523585647, 0.0, 77.56161960922043,
       0.0, -0.00026949458523585647, 31.290207807979588)}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 12917, 'height': 9551, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00026949458523585647, 0.0, 77.56161960922043,
       0.0, -0.00026949458523585647, 31.290207807979588)}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 12917, 'height': 9551, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00026949458523585647, 0.0, 77.56161960922043,
       0.0, -0.00026949458523585647, 31.290207807

In [6]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('resample_log.txt'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [7]:
# @title Create Stacked Image
weather_files = sorted(glob.glob(f"{weather_dir}/era5_2016_*.tif"))

nodata_value = -9999

if not weather_files:
    print("No weather files found!")
    exit()

# Get list of date strings
date_strs = [os.path.basename(wf).replace('era5_2016_', '').replace('.tif', '') for wf in weather_files]
logger.info(f"Found {len(date_strs)} weather files to process.")

# Loop to process data
for date_str in date_strs:
    weather_path = os.path.join(weather_dir, f'era5_2016_{date_str}.tif')
    fire_path = os.path.join(fire_mask_dir, f'fire_mask_2016_{date_str}.tif')

    logger.info(f"Processing date: {date_str}")

    if not os.path.exists(weather_path):
        logger.warning(f"Skipping {date_str}: weather missing at {weather_path}")
        continue

    if not os.path.exists(fire_path):
        logger.warning(f"Skipping {date_str}: fire mask missing at {fire_path}")
        continue

    # Load weather data (5 bands)
    try:
        logger.info(f"Loading weather data: {weather_path}")
        with rasterio.open(weather_path) as w_src:
            weather = w_src.read()  # Read all 5 bands
            if weather.shape[0] != 5 or weather.shape[1:] != ref_shape:
                raise ValueError(f"Weather data shape {weather.shape} does not match reference shape {ref_shape}")
            logger.info(f"Successfully loaded weather data: {weather_path}")
    except Exception as e:
        logger.error(f"Failed to load resampled weather data for {date_str}: {str(e)}")
        continue

    # Load fire mask (1 band)
    try:
        logger.info(f"Loading fire mask: {fire_path}")
        with rasterio.open(fire_path) as fm_src:
            fire = fm_src.read(1)  # Read single band
            if fire.shape != ref_shape:
                raise ValueError(f"Fire mask shape {fire.shape} does not match reference shape {ref_shape}")
            logger.info(f"Successfully loaded fire mask: {fire_path}")
    except Exception as e:
        logger.error(f"Failed to load resampled fire mask for {date_str}: {str(e)}")
        continue

    # Stack and save with logging
    try:
        logger.info(f"Stacking data for date: {date_str}")

        # Fill masked arrays
        slope_filled = slope.filled(nodata_value)
        aspect_filled = aspect.filled(nodata_value)
        fuel_filled = fuel.filled(nodata_value)
        ghsl_filled = ghsl.filled(nodata_value)
        fire_filled = fire  # Already a regular array

        out_path = os.path.join(output_dir, f'stack_2016_{date_str}.tif')
        
        # Stack into one array (10 bands)
        stacked = np.stack([
            *weather, # 1-5 Bands
            slope_filled, # 6 Band
            aspect_filled, # 7 Band
            fuel_filled, # 8 Band
            ghsl_filled, # 9 Band
            fire_filled # 10 Band
        ])

        # Update metadata
        ref_meta.update({
            'count': 10,
            'dtype': 'float32',
            'compress': 'deflate',     # Use DEFLATE
            'predictor': 2,            # Helps float compression
            'zlevel': 6,                # Compression level (1–9), 6 = balanced
            "nodata": nodata_value
        })

        # Save
        with rasterio.open(out_path, 'w', **ref_meta) as dst:
            dst.write(stacked.astype('float32'))

        logger.info(f"Successfully saved stacked raster: {out_path}")
        print(f"✅ Saved: {out_path}")
    except Exception as e:
        logger.error(f"Failed to stack or save data for {date_str}: {str(e)}")
        continue

    w_src.close()
    fm_src.close()

s.close()
a.close()
f.close()
g.close()

logger.info("Processing complete.")

2025-07-03 16:47:02,051 - INFO - Found 59 weather files to process.
2025-07-03 16:47:02,052 - INFO - Processing date: 04_01
2025-07-03 16:47:02,054 - INFO - Loading weather data: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/weather/era5_2016_04_01.tif
2025-07-03 16:47:07,431 - INFO - Successfully loaded weather data: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/weather/era5_2016_04_01.tif
2025-07-03 16:47:07,540 - INFO - Loading fire mask: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/fire_mask/fire_mask_2016_04_01.tif
2025-07-03 16:47:07,666 - INFO - Successfully loaded fire mask: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/fire_mask/fire_mask_2016_04_01.tif
2025-07-03 16:47:07,680 - INFO - Stacking data for date: 04_01
2025-07-03 16:47:52,321 - INFO - Successfully saved stacked raster: /home/swayam/projects/forest_fire_spread/datasets/dataset_stacked/stack_2016_04_01.tif
2025-07-03 16:47:52,32

✅ Saved: /home/swayam/projects/forest_fire_spread/datasets/dataset_stacked/stack_2016_04_01.tif


2025-07-03 16:47:57,178 - INFO - Successfully loaded weather data: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/weather/era5_2016_04_02.tif
2025-07-03 16:47:57,182 - INFO - Loading fire mask: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/fire_mask/fire_mask_2016_04_02.tif
2025-07-03 16:47:57,246 - INFO - Successfully loaded fire mask: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/fire_mask/fire_mask_2016_04_02.tif
2025-07-03 16:47:57,249 - INFO - Stacking data for date: 04_02
2025-07-03 16:48:40,574 - INFO - Successfully saved stacked raster: /home/swayam/projects/forest_fire_spread/datasets/dataset_stacked/stack_2016_04_02.tif
2025-07-03 16:48:40,575 - INFO - Processing date: 04_03
2025-07-03 16:48:40,576 - INFO - Loading weather data: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/weather/era5_2016_04_03.tif


✅ Saved: /home/swayam/projects/forest_fire_spread/datasets/dataset_stacked/stack_2016_04_02.tif


2025-07-03 16:48:45,151 - INFO - Successfully loaded weather data: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/weather/era5_2016_04_03.tif
2025-07-03 16:48:45,154 - INFO - Loading fire mask: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/fire_mask/fire_mask_2016_04_03.tif
2025-07-03 16:48:45,308 - INFO - Successfully loaded fire mask: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/fire_mask/fire_mask_2016_04_03.tif
2025-07-03 16:48:45,311 - INFO - Stacking data for date: 04_03
2025-07-03 16:49:28,220 - INFO - Successfully saved stacked raster: /home/swayam/projects/forest_fire_spread/datasets/dataset_stacked/stack_2016_04_03.tif
2025-07-03 16:49:28,221 - INFO - Processing date: 04_04
2025-07-03 16:49:28,221 - INFO - Loading weather data: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/weather/era5_2016_04_04.tif


...

✅ Saved: /home/swayam/projects/forest_fire_spread/datasets/dataset_stacked/stack_2016_05_27.tif


2025-07-03 17:34:31,341 - INFO - Successfully loaded weather data: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/weather/era5_2016_05_28.tif
2025-07-03 17:34:31,344 - INFO - Loading fire mask: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/fire_mask/fire_mask_2016_05_28.tif
2025-07-03 17:34:31,510 - INFO - Successfully loaded fire mask: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/fire_mask/fire_mask_2016_05_28.tif
2025-07-03 17:34:31,513 - INFO - Stacking data for date: 05_28
2025-07-03 17:35:16,918 - INFO - Successfully saved stacked raster: /home/swayam/projects/forest_fire_spread/datasets/dataset_stacked/stack_2016_05_28.tif
2025-07-03 17:35:16,919 - INFO - Processing date: 05_29
2025-07-03 17:35:16,921 - INFO - Loading weather data: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/weather/era5_2016_05_29.tif


✅ Saved: /home/swayam/projects/forest_fire_spread/datasets/dataset_stacked/stack_2016_05_28.tif


2025-07-03 17:35:21,778 - INFO - Successfully loaded weather data: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/weather/era5_2016_05_29.tif
2025-07-03 17:35:21,781 - INFO - Loading fire mask: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/fire_mask/fire_mask_2016_05_29.tif
2025-07-03 17:35:21,953 - INFO - Successfully loaded fire mask: /home/swayam/projects/forest_fire_spread/datasets/dataset_unstacked/fire_mask/fire_mask_2016_05_29.tif
2025-07-03 17:35:21,956 - INFO - Stacking data for date: 05_29
2025-07-03 17:36:06,111 - INFO - Successfully saved stacked raster: /home/swayam/projects/forest_fire_spread/datasets/dataset_stacked/stack_2016_05_29.tif
2025-07-03 17:36:06,133 - INFO - Processing complete.


✅ Saved: /home/swayam/projects/forest_fire_spread/datasets/dataset_stacked/stack_2016_05_29.tif
